In [ ]:
import random
import math
import random
from sympy import isprime
import sys
import os

# Function generates a list of positive integers e such that each value is greater than the sum of prev ones.
# @param n: the length of the sequence
# @returns e_sequence: part of the private key
def generate_e_sequence(n):
    # Start list with a random integer. 
    # SystemRandom() was used since it's most secure, as it relies on underlying OS mechanisms.
    # Upperbound was system size since it is very large for increased security & randomness
    # (on a 64-bit system, it's 2^63 − 1)
    e_sequence = [random.SystemRandom().randint(1,  sys.maxsize/n)]

    for i in range(1, n):
        # Set the minimum value of the next element to be greater than the sum of all previous elements
        min_value = sum(e_sequence) + 1
        # Randomly select a value greater than the sum of previous elements
        # Upperbound chosen as twice the size of min_value so that random values don't go out of bounds
        e_sequence.append(random.randint(min_value, min_value * 2))

    return e_sequence

# function finds a random prime number greater than a specified number n.
# @param n: number to find random primes number greater than 
def find_random_prime_greater_than(n):
    while True:
        # Generate a random number greater than n. 
        potential_prime = random.SystemRandom().randint(n+1, n+sys.maxsize)
        # Return random number if it's prime 
        if isprime(potential_prime):
            return potential_prime

# Function generates public key and part of the private key (q and w)  
# @param e_sequence: list of integers with random values 
# @returns q: prime number at least twice as large as the max value in e_sequence
# @returns w: number relatively prime to q
# @returns h_sequence: public key
def select_prime_and_compute_h(e_sequence):
    # Finds prime number greater than twice max of e_sequence (i.e., the last value)
    q = find_random_prime_greater_than(2 * max(e_sequence))
    
    # Loops until a w that is relatively prime to q is found
    while True:
        w = random.SystemRandom().randint(2, sys.maxsize)
        if math.gcd(w, q) == 1 and w != q:
            break    
    
    # Compute public key
    h_sequence = [(w * e) % q for e in e_sequence]

    return q, w, h_sequence

# Function splits message into blocks then encrypts it using the public key, h_sequence. 
# @param message: plaintext to be encrypted
# @param h_sequence: public key which is a sequence of integers
# @returns encrypted_blocks: list of integers, which are encrypted blocks of the original message.
def encrypt_message(message, h_sequence):   
    # Convert the message to a binary string
    binary_str = ''.join(format(ord(char), '08b') for char in message)
    
    # Calculate the length of each block
    block_length = len(h_sequence)

    # Split the binary string into blocks of size block_length
    blocks = [binary_str[i:i + block_length] for i in range(0, len(binary_str), block_length)]

    # list for encrypted blocks
    encrypted_blocks = []

    # Encrypt each block
    for block in blocks:
        c = 0
        # sets length of the current block (useful if the last block is shorter than block_length)
        min_length = min(len(h_sequence), len(block))
        # Iterates through each bit of the block and the corresponding value in h_sequence
        for i in range(min_length):
            c += int(block[i]) * h_sequence[i]
        # Appends the encrypted value of the block
        encrypted_blocks.append(c)

    return encrypted_blocks

# Function decrypts a list of encrypted integer blocks using the private key (e_sequence, q, w).
# @param encrypted_blocks: list of integers which are encrypted blocks of original message
# @param e_sequence: list of integers part of the private key
# @param q: integer part of the private key, used in the modular arithmetic during decryption.
# @param w: integer part of the private key, used to calculate the modular multiplicative inverse.
# @returns decrypted_message: decrypted message as a string
def decrypt_message(encrypted_blocks, e_sequence, q, w):
    # Initialize empty string to hold decrypted message
    decrypted_message = ''
    
    # Compute the modular multiplicative inverse of w with respect to q
    w_inv = pow(w, -1, q)
    
    # Iterate through each encrypted block
    for c in encrypted_blocks:
        # c' is the product of the encrypted block and the modular inverse of w, modulo q
        c_prime = (c * w_inv) % q

        # Initialize empty string for the binary representation of decrypted block
        decrypted_binary_str = ''
        # Iterate through the elements of e_sequence starting from the largest
        for e in reversed(e_sequence):
            # If c' is greater than or equal to the current element, add '1' to the binary string and reduce c' by that element
            if c_prime >= e:
                decrypted_binary_str = '1' + decrypted_binary_str
                c_prime -= e
            # If c' is less than the current element, add '0' to the binary string
            else:
                decrypted_binary_str = '0' + decrypted_binary_str

        # Convert the binary string to text and append to the decrypted message
        decrypted_text = ''.join(chr(int(decrypted_binary_str[i:i+8], 2)) for i in range(0, len(decrypted_binary_str), 8))
        decrypted_message += decrypted_text
    
    # Return fully decrypted message
    return decrypted_message

# Function generates a new filename based on the original filename and the operation.
# @param filename: filename as a string
# @param operation: string indicating the operation, '-e' for encryption and '-d' for decryption
# @returns new filename as string with a suffix based on the operation
def generate_output_filename(filename, operation):
    # Split filename into the name and extension
    name, ext = os.path.splitext(filename)
    # Determine suffix based on the operation
    suffix = '_encrypted' if operation == '-e' else '_decrypted'
    return f"{name}{suffix}{ext}"

# Function prompts user for file and operation (encryption/decryption) and processes message based on that 
def process_message():
    # Continuously prompt for user input until valid input is received
    while True:
        # Prompt user to enter the filename and operation (encryption or decryption) using appropriate flag
        user_input = input("Enter your text filename followed by -e for encryption or -d for decryption (e.g., 'message.txt -e'):\n ")
        # Split the input into filename and operation
        parts = user_input.split()

         # Check if the format of the input is correct
        if len(parts) != 2 or parts[1] not in ['-e', '-d']:
            print("Invalid input format. Please try again!")
            continue

        # Unpack the filename and operation from the input
        filename, operation = parts

        # Check if the file is a .txt file
        if not filename.endswith('.txt'):
            print("Please provide a .txt file. Other file types are not supported.")
            continue
            
        #Check if specified file exists
        if not os.path.exists(filename):
            print("File does not exist. Please try again!")
            continue
            
        # Exits loop if valid input is received
        break

    # Open and read the contents of the file into message
    with open(filename, 'r') as file:
        message = file.read()
    
    # Generate a new filename based on the operation
    output_filename = generate_output_filename(filename, operation)

    # Iterate until public/private key entered correctly
    while True:
        try:
            # Process message based on the operation
            if operation == '-e':
                # Prompt for public key to encrypt message
                h_sequence_input = input("Enter the public key (h_sequence) as comma-separated values:\n").replace(" ", "")
                # Convert public key into list of integers from string sequence
                h_sequence = [int(x.strip()) for x in h_sequence_input.split(',')]
                # Encrypt the message using the public key
                processed_message = encrypt_message(message, h_sequence)  
                # Convert the encrypted blocks to a space-separated string
                processed_message_str = ' '.join(map(str, processed_message))  
                break
                
            elif operation == '-d':
                # Prompt for private key to decrypt message and parse to remove any spaces
                private_key_input = input("Enter the private key (e_sequence, q, w) as comma-separated values (e.g., [1,2,4],45,47):\n").replace(" ", "")

                # Parse e_sequence, q, and w from the input
                closing_bracket_index = private_key_input.find(']')
                e_sequence_str = private_key_input[1:closing_bracket_index]
                q_str, w_str = private_key_input[closing_bracket_index + 2:].split(',', 1)

                # Convert e_sequence_str to a list of integers
                e_sequence = [int(x.strip()) for x in e_sequence_str.split(',')]

                # Convert q_str and w_str to integers
                q = int(q_str.strip())
                w = int(w_str.strip())

                # converts contents of message file which are encrypted blocks seperated by space 
                # (if the same program was used for encryption) to list of integers
                encrypted_blocks = [int(x) for x in message.split()]
                
                # Decrypt the message using the private key
                processed_message = decrypt_message(encrypted_blocks, e_sequence, q, w)
                processed_message_str = processed_message
                break 
        
        # If invalid key format used, prompt user again
        except ValueError:
            print("Invalid key format. Please try again!")
            continue

    # Write the processed message to the output file
    with open(output_filename, 'w') as file:
        file.write(processed_message_str)
        
    # Notify the user of output filename
    print(f"Processed file saved as: {output_filename}")

def main():
    # Prompt user for input until they terminate by inputting 't'
    while True:
        # Ask user if they want to generate a key pair, encrpt/decrypt a file or terminate session
        user_choice = input("\nEnter '1' to generate a public/private key pair or\nEnter '2' to encrypt/decrypt a file or\nEnter 't' to terminate session:)\n")
        
        # If user wants to generate key pair
        if user_choice == '1':
            # Ask user if they want a short/long key
            key_size_choice = input("Enter 's' for a short key length ideal for testing (size 8) or\nEnter 'l' for a more secure key (size 256):\n")

            # Assign key size based on user's choice
            if key_size_choice == 's':
                key_size = 8
            elif key_size_choice == 'l':
                key_size = 216
            else:
                # If invalid key size is inputted, alert user and reprompt them
                print("Invalid key size choice. Please try again!")
                continue

            # Generate the public/private key pair
            e_sequence = generate_e_sequence(key_size)
            q, w, h_sequence = select_prime_and_compute_h(e_sequence)
            
            # Display the keys 
            print("Public/Private Key Pair Generated :)")
            print("Private Key (e_sequence, q, w):", e_sequence,",", q,",", w)
            print("\nPublic Key (h_sequence):", ', '.join(map(str, h_sequence)))
            
        # If user wants to encrypt/decrypt a message
        elif user_choice == '2':
            # function processes users message by encrypting/decrypting as they choose
            process_message()
            
        # If user wants to terminate session
        elif user_choice == 't':
            print("Session Terminated :)")
            # Exit loop since session terminated
            break  
        # Handle any other invalid input
        else:
            print("Invalid input. Please try again!")

main()


Enter '1' to generate a public/private key pair or
Enter '2' to encrypt/decrypt a file or
Enter 't' to terminate session:)
2
Enter your text filename followed by -e for encryption or -d for decryption (e.g., 'message.txt -e'):
 hello.txt
Invalid input format. Please try again!
Enter your text filename followed by -e for encryption or -d for decryption (e.g., 'message.txt -e'):
 hello.txt -e
Enter the public key (h_sequence) as comma-separated values:
86073308652546949776, 151559335921230004554, 180931658915026927240, 119174810736936443758, 88769448387469800293, 1569953384568833376, 214212878107218490677, 248462887117781150856
Processed file saved as: hello_encrypted.txt


In [1]:
import random
import math
import random
from sympy import isprime
import sys
import os

In [2]:
# Function generates a list of positive integers e such that each value is greater than the sum of prev ones.
# @param n: the length of the sequence
# @returns e_sequence: part of the private key
def generate_e_sequence(n):
    # Start list with a random integer. 
    # SystemRandom() was used since it's most secure, as it relies on underlying OS mechanisms.
    # Upperbound was system size since it is very large for increased security & randomness
    # (on a 64-bit system, it's 2^63 − 1)
    e_sequence = [random.SystemRandom().randint(1,  sys.maxsize/n)]

    for i in range(1, n):
        # Set the minimum value of the next element to be greater than the sum of all previous elements
        min_value = sum(e_sequence) + 1
        # Randomly select a value greater than the sum of previous elements
        # Upperbound chosen as twice the size of min_value so that random values don't go out of bounds
        e_sequence.append(random.randint(min_value, min_value * 2))

    return e_sequence

In [3]:
# function finds a random prime number greater than a specified number n.
# @param n: number to find random primes number greater than 
def find_random_prime_greater_than(n):
    while True:
        # Generate a random number greater than n. 
        potential_prime = random.SystemRandom().randint(n+1, n+sys.maxsize)
        # Return random number if it's prime 
        if isprime(potential_prime):
            return potential_prime

In [4]:
# Function generates public key and part of the private key (q and w)  
# @param e_sequence: list of integers with random values 
# @returns q: prime number at least twice as large as the max value in e_sequence
# @returns w: number relatively prime to q
# @returns h_sequence: public key
def select_prime_and_compute_h(e_sequence):
    # Finds prime number greater than twice max of e_sequence (i.e., the last value)
    q = find_random_prime_greater_than(2 * max(e_sequence))
    
    # Loops until a w that is relatively prime to q is found
    while True:
        w = random.SystemRandom().randint(2, sys.maxsize)
        if math.gcd(w, q) == 1 and w != q:
            break    
    
    # Compute public key
    h_sequence = [(w * e) % q for e in e_sequence]

    return q, w, h_sequence

In [5]:
# Function splits message into blocks then encrypts it using the public key, h_sequence. 
# @param message: plaintext to be encrypted
# @param h_sequence: public key which is a sequence of integers
# @returns encrypted_blocks: list of integers, which are encrypted blocks of the original message.
def encrypt_message(message, h_sequence):   
    # Convert the message to a binary string
    binary_str = ''.join(format(ord(char), '08b') for char in message)
    
    # Calculate the length of each block
    block_length = len(h_sequence)

    # Split the binary string into blocks of size block_length
    blocks = [binary_str[i:i + block_length] for i in range(0, len(binary_str), block_length)]

    # list for encrypted blocks
    encrypted_blocks = []

    # Encrypt each block
    for block in blocks:
        c = 0
        # sets length of the current block (useful if the last block is shorter than block_length)
        min_length = min(len(h_sequence), len(block))
        # Iterates through each bit of the block and the corresponding value in h_sequence
        for i in range(min_length):
            c += int(block[i]) * h_sequence[i]
        # Appends the encrypted value of the block
        encrypted_blocks.append(c)

    return encrypted_blocks

In [6]:
# Function decrypts a list of encrypted integer blocks using the private key (e_sequence, q, w).
# @param encrypted_blocks: list of integers which are encrypted blocks of original message
# @param e_sequence: list of integers part of the private key
# @param q: integer part of the private key, used in the modular arithmetic during decryption.
# @param w: integer part of the private key, used to calculate the modular multiplicative inverse.
# @returns decrypted_message: decrypted message as a string
def decrypt_message(encrypted_blocks, e_sequence, q, w):
    # Initialize empty string to hold decrypted message
    decrypted_message = ''
    
    # Compute the modular multiplicative inverse of w with respect to q
    w_inv = pow(w, -1, q)
    
    # Iterate through each encrypted block
    for c in encrypted_blocks:
        # c' is the product of the encrypted block and the modular inverse of w, modulo q
        c_prime = (c * w_inv) % q

        # Initialize empty string for the binary representation of decrypted block
        decrypted_binary_str = ''
        # Iterate through the elements of e_sequence starting from the largest
        for e in reversed(e_sequence):
            # If c' is greater than or equal to the current element, add '1' to the binary string and reduce c' by that element
            if c_prime >= e:
                decrypted_binary_str = '1' + decrypted_binary_str
                c_prime -= e
            # If c' is less than the current element, add '0' to the binary string
            else:
                decrypted_binary_str = '0' + decrypted_binary_str

        # Convert the binary string to text and append to the decrypted message
        decrypted_text = ''.join(chr(int(decrypted_binary_str[i:i+8], 2)) for i in range(0, len(decrypted_binary_str), 8))
        decrypted_message += decrypted_text
    
    # Return fully decrypted message
    return decrypted_message

In [7]:
# Function generates a new filename based on the original filename and the operation.
# @param filename: filename as a string
# @param operation: string indicating the operation, '-e' for encryption and '-d' for decryption
# @returns new filename as string with a suffix based on the operation
def generate_output_filename(filename, operation):
    # Split filename into the name and extension
    name, ext = os.path.splitext(filename)
    # Determine suffix based on the operation
    suffix = '_encrypted' if operation == '-e' else '_decrypted'
    return f"{name}{suffix}{ext}"

In [8]:
# Function prompts user for file and operation (encryption/decryption) and processes message based on that 
def process_message():
    # Continuously prompt for user input until valid input is received
    while True:
        # Prompt user to enter the filename and operation (encryption or decryption) using appropriate flag
        user_input = input("Enter your text filename followed by -e for encryption or -d for decryption (e.g., 'message.txt -e'): ")
        # Split the input into filename and operation
        parts = user_input.split()

         # Check if the format of the input is correct
        if len(parts) != 2 or parts[1] not in ['-e', '-d']:
            print("Invalid input format. Please try again!")
            continue

        # Unpack the filename and operation from the input
        filename, operation = parts

        # Check if the file is a .txt file
        if not filename.endswith('.txt'):
            print("Please provide a .txt file. Other file types are not supported.")
            continue
            
        #Check if specified file exists
        if not os.path.exists(filename):
            print("File does not exist. Please try again!")
            continue
            
        # Exits loop if valid input is received
        break

    # Open and read the contents of the file into message
    with open(filename, 'r') as file:
        message = file.read()
    
    # Generate a new filename based on the operation
    output_filename = generate_output_filename(filename, operation)

    # Iterate until public/private key entered correctly
    while True:
        try:
            # Process message based on the operation
            if operation == '-e':
                # Prompt for public key to encrypt message
                h_sequence_input = input("Enter the public key (h_sequence) as comma-separated values: ").replace(" ", "")
                # Convert public key into list of integers from string sequence
                h_sequence = [int(x.strip()) for x in h_sequence_input.split(',')]
                # Encrypt the message using the public key
                processed_message = encrypt_message(message, h_sequence)  
                # Convert the encrypted blocks to a space-separated string
                processed_message_str = ' '.join(map(str, processed_message))  
                break
                
            elif operation == '-d':
                # Prompt for private key to decrypt message and parse to remove any spaces
                private_key_input = input("Enter the private key (e_sequence, q, w) as comma-separated values (e.g., [1,2,4],45,47): ").replace(" ", "")

                # Parse e_sequence, q, and w from the input
                closing_bracket_index = private_key_input.find(']')
                e_sequence_str = private_key_input[1:closing_bracket_index]
                q_str, w_str = private_key_input[closing_bracket_index + 2:].split(',', 1)

                # Convert e_sequence_str to a list of integers
                e_sequence = [int(x.strip()) for x in e_sequence_str.split(',')]

                # Convert q_str and w_str to integers
                q = int(q_str.strip())
                w = int(w_str.strip())

                # converts contents of message file which are encrypted blocks seperated by space 
                # (if the same program was used for encryption) to list of integers
                encrypted_blocks = [int(x) for x in message.split()]
                
                # Decrypt the message using the private key
                processed_message = decrypt_message(encrypted_blocks, e_sequence, q, w)
                processed_message_str = processed_message
                break 
        
        # If invalid key format used, prompt user again
        except ValueError:
            print("Invalid key format. Please try again!")
            continue

    # Write the processed message to the output file
    with open(output_filename, 'w') as file:
        file.write(processed_message_str)
        
    # Notify the user of output filename
    print(f"Processed file saved as: {output_filename}")

In [9]:
def main():
    # Prompt user for input until they terminate by inputting 't'
    while True:
        # Ask user if they want to generate a key pair, encrpt/decrypt a file or terminate session
        user_choice = input("\nEnter '1' to generate a public/private key pair or\nEnter '2' to encrypt/decrypt a file or\nEnter 't' to terminate session:)\n")
        
        # If user wants to generate key pair
        if user_choice == '1':
            # Ask user if they want a short/long key
            key_size_choice = input("Enter 's' for a short key length ideal for testing (size 8) or\nEnter 'l' for a more secure key (size 256):\n")

            # Assign key size based on user's choice
            if key_size_choice == 's':
                key_size = 8
            elif key_size_choice == 'l':
                key_size = 216
            else:
                # If invalid key size is inputted, alert user and reprompt them
                print("Invalid key size choice. Please try again!")
                continue

            # Generate the public/private key pair
            e_sequence = generate_e_sequence(key_size)
            q, w, h_sequence = select_prime_and_compute_h(e_sequence)
            
            # Display the keys 
            print("Public/Private Key Pair Generated :)")
            print("Private Key (e_sequence, q, w):", e_sequence,",", q,",", w)
            print("\nPublic Key (h_sequence):", ', '.join(map(str, h_sequence)))
            
        # If user wants to encrypt/decrypt a message
        elif user_choice == '2':
            # function processes users message by encrypting/decrypting as they choose
            process_message()
            
        # If user wants to terminate session
        elif user_choice == 't':
            print("Session Terminated :)")
            # Exit loop since session terminated
            break  
        # Handle any other invalid input
        else:
            print("Invalid input. Please try again!")

main()


Enter '1' to generate a public/private key pair or
Enter '2' to encrypt/decrypt a file or
Enter 't' to terminate session:)
1
Enter 's' for a short key length ideal for testing (size 8) or
Enter 'l' for a more secure key (size 256):
s
Public/Private Key Pair Generated :)
Private Key (e_sequence, q, w): [1119108904039568321, 1167897499313597013, 3373616292677582782, 9442263440691273427, 17729489888038834043, 64093881394657202821, 134948964880102347475, 318502750615368070636] , 637155707943050040337 , 626921389258559184

Public Key (h_sequence): 573048654129928042307, 205654067643566334463, 360836174384429549547, 617611446284645477077, 45715294050892361140, 515812627865573611842, 80433117461611114192, 51252912772038186031

Enter '1' to generate a public/private key pair or
Enter '2' to encrypt/decrypt a file or
Enter 't' to terminate session:)
2
Enter your text filename followed by -e for encryption or -d for decryption (e.g., 'message.txt -e'): hello.txt -e
Enter the public key (h_seque